In [ ]:
import os, torch

# Where HF will cache stuff in general (models + datasets)
os.environ["HF_HOME"] = r"D:\hf_cache"
os.makedirs(os.environ["HF_HOME"], exist_ok=True)

# Where we keep THIS project’s model (base + fine-tuned)
MODEL_DIR = r"D:\hf_models\SarcasmDetector"
os.makedirs(MODEL_DIR, exist_ok=True)

print("HF cache:", os.environ["HF_HOME"])
print("Model dir:", MODEL_DIR)

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


HF cache: D:\hf_cache
Model dir: D:\hf_models\SarcasmDetector
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

base_model_name = "roberta-base"

# Download from HF hub (first time only)…
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=2  # SARCASM vs NOT_SARCASM
)

# …and save to your permanent folder
tokenizer.save_pretrained(MODEL_DIR)
model.save_pretrained(MODEL_DIR)

print("Base model downloaded and saved to:", MODEL_DIR)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base model downloaded and saved to: D:\hf_models\roberta_sarcasm


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Loaded model from:", MODEL_DIR)


Loaded model from: D:\hf_models\roberta_sarcasm


In [ ]:
from datasets import load_dataset, Features, Value, Sequence

# 1. Direct URLs to the 4 jsonl files on Hugging Face
data_files = {
    "train": [
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_reddit_training.jsonl",
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_twitter_training.jsonl",
    ],
    "test": [
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_reddit_testing.jsonl",
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_twitter_testing.jsonl",
    ],
}

# 2. Explicit schema that *includes* the optional 'id' column
features = Features(
    {
        "label":   Value("string"),
        "response": Value("string"),
        "context": Sequence(Value("string")),
        "id":      Value("string"),  # some files have it, some don't -> will be None
    }
)

dataset = load_dataset(
    "json",
    data_files=data_files,
    features=features,
)

print(dataset)
print(dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['label', 'response', 'context', 'id'],
        num_rows: 9400
    })
    test: Dataset({
        features: ['label', 'response', 'context', 'id'],
        num_rows: 3600
    })
})
{'label': 'SARCASM', 'response': "Yeah I mean there's only one gender anyways, women are objects", 'context': ['LPT: If you\'re worried about hurting someone\'s feelings by using the incorrect gender pronoun, play it safe and spare them any unnecessary pain by using the neutral word "it"', 'When gender is unknown he/him is default.'], 'id': None}


In [ ]:
from transformers import AutoTokenizer
# assuming `tokenizer` already loaded before this

label2id = {"NOT_SARCASM": 0, "SARCASM": 1}
id2label = {v: k for k, v in label2id.items()}
model.config.label2id = label2id
model.config.id2label = id2label

def build_text(example):
    ctx = " [CTX] ".join(example["context"]) if example["context"] else ""
    if ctx:
        full = f"{ctx} [RESP] {example['response']}"
    else:
        full = example["response"]
    return full

def preprocess(example):
    text = build_text(example)
    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    enc["labels"] = label2id[example["label"]]
    return enc

encoded = dataset.map(preprocess)

# keep only tensor columns (drop label/context/response/id)
cols_to_keep = ["input_ids", "attention_mask", "labels"]
encoded = encoded.remove_columns(
    [c for c in encoded["train"].column_names if c not in cols_to_keep]
)
encoded.set_format(type="torch")

train_ds = encoded["train"]
test_ds  = encoded["test"]
print(train_ds[0])


Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

{'input_ids': tensor([    0,   574, 10311,    35,   318,    47,   214,  3915,    59, 12780,
          951,    18,  6453,    30,   634,     5, 17401,  3959, 44811,     6,
          310,    24,  1522,     8, 10628,   106,   143, 10495,  2400,    30,
          634,     5,  7974,  2136,    22,   405,   113,   646,  7164,  1000,
          742,   520,  3959,    16,  4727,    37,    73, 24770,    16,  6814,
            4,   646, 32030,   510,   742,  8976,    38,  1266,    89,    18,
          129,    65,  3959, 42563,     6,   390,    32,  8720,     2,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_DIR, "checkpoints"),
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,

    # ✅ new name in recent transformers
    eval_strategy="epoch",          # instead of evaluation_strategy
    save_strategy="epoch",
    logging_strategy="epoch",       # optional, but nice to keep in sync
    logging_steps=100,

    load_best_model_at_end=True,
    fp16=True,                      # good for your 4070
)


In [ ]:
from transformers import Trainer
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.596100,0.647567,0.599444
2,0.477800,0.676947,0.621389
3,0.371500,0.786351,0.635278


TrainOutput(global_step=1764, training_loss=0.4818017142159598, metrics={'train_runtime': 211.5299, 'train_samples_per_second': 133.314, 'train_steps_per_second': 8.339, 'total_flos': 1854932940288000.0, 'train_loss': 0.4818017142159598, 'epoch': 3.0})

In [ ]:
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

print("Fine-tuned model saved to:", MODEL_DIR)


Fine-tuned model saved to: D:\hf_models\roberta_sarcasm


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

MODEL_DIR = r"D:\hf_models\roberta_sarcasm"  # same dir

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True
).to("cuda")

label_names = ["NOT_SARCASM", "SARCASM"]

def predict_sarcasm(text, context=None):
    if context is None:
        context = []
    if isinstance(context, str):
        context = [context]
    ctx = " [CTX] ".join(context)
    full = f"{ctx} [RESP] {text}" if ctx else text

    inputs = tokenizer(full, return_tensors="pt", truncation=True,
                       padding=True, max_length=128).to("cuda")
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)[0].cpu().numpy()

    pred_id = int(probs.argmax())
    return {
        "label": label_names[pred_id],
        "probs": {label_names[i]: float(p) for i, p in enumerate(probs)}
    }

print(predict_sarcasm(
    "That is a good one",
    context=["It"]
))


{'label': 'SARCASM', 'probs': {'NOT_SARCASM': 0.025513367727398872, 'SARCASM': 0.9744866490364075}}


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_DIR = r"D:\hf_models\roberta_sarcasm"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True
).to(device)

model.eval()
print("Loaded model from:", MODEL_DIR)
print("id2label:", model.config.id2label)


Loaded model from: D:\hf_models\roberta_sarcasm
id2label: {0: 'NOT_SARCASM', 1: 'SARCASM'}


In [ ]:
from datasets import load_dataset

# "irony" sub-task of tweet_eval
raw_test = load_dataset("tweet_eval", "irony", split="test")
print(raw_test)
print(raw_test[0])


README.md: 0.00B [00:00, ?B/s]

irony/train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

irony/test-00000-of-00001.parquet:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

irony/validation-00000-of-00001.parquet:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2862 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 784
})
{'text': '@user Can U Help?||More conservatives needed on #TSU + get paid 4 posting stuff like this!||YOU $ can go to', 'label': 0}


In [ ]:
def preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

test_ds = raw_test.map(preprocess, batched=True)

# rename label -> labels for PyTorch
test_ds = test_ds.rename_column("label", "labels")

# keep only the tensor columns
cols = ["input_ids", "attention_mask", "labels"]
test_ds.set_format(type="torch", columns=cols)

print(test_ds[0])


Map:   0%|          | 0/784 [00:00<?, ? examples/s]

{'labels': tensor(0), 'input_ids': tensor([    0,  1039, 12105,  2615,   121, 10310,   116, 49085,  9690,  9930,
          956,    15,   849,  2685,   791,  2055,   120,  1199,   204,  6016,
         2682,   101,    42,   328, 49085, 38158,    68,    64,   213,     7,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1, 

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

batch_size = 32
loader = DataLoader(test_ds, batch_size=batch_size)

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].cpu().numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.cpu().numpy()
        preds = np.argmax(logits, axis=-1)

        all_preds.extend(preds)
        all_labels.extend(labels)

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

acc = accuracy_score(all_labels, all_preds)
print(f"\nAccuracy on tweet_eval/irony test set: {acc:.4f}\n")

print("Classification report:")
print(classification_report(all_labels, all_preds,
                            target_names=["NOT_SARCASM", "SARCASM"]))

print("Confusion matrix:")
print(confusion_matrix(all_labels, all_preds))



Accuracy on tweet_eval/irony test set: 0.7883

Classification report:
              precision    recall  f1-score   support

 NOT_SARCASM       0.86      0.78      0.82       473
     SARCASM       0.70      0.81      0.75       311

    accuracy                           0.79       784
   macro avg       0.78      0.79      0.78       784
weighted avg       0.80      0.79      0.79       784

Confusion matrix:
[[367 106]
 [ 60 251]]


More Tuning

In [ ]:
from datasets import load_dataset, Features, Value, Sequence

data_files = {
    "train": [
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_reddit_training.jsonl",
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_twitter_training.jsonl",
    ],
    "test": [
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_reddit_testing.jsonl",
        "https://huggingface.co/datasets/tasksource/figlang2020-sarcasm/resolve/main/sarcasm_detection_shared_task_twitter_testing.jsonl",
    ],
}

features = Features(
    {
        "label":   Value("string"),
        "response": Value("string"),
        "context": Sequence(Value("string")),
        "id":      Value("string"),
    }
)

fig = load_dataset("json", data_files=data_files, features=features)
print(fig)


(…)ection_shared_task_reddit_training.jsonl: 0.00B [00:00, ?B/s]

(…)ction_shared_task_twitter_training.jsonl: 0.00B [00:00, ?B/s]

(…)tection_shared_task_reddit_testing.jsonl: 0.00B [00:00, ?B/s]

(…)ection_shared_task_twitter_testing.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'response', 'context', 'id'],
        num_rows: 9400
    })
    test: Dataset({
        features: ['label', 'response', 'context', 'id'],
        num_rows: 3600
    })
})


In [ ]:
def make_text(example):
    ctx = " [CTX] ".join(example["context"]) if example["context"] else ""
    example["text"] = f"{ctx} [RESP] {example['response']}" if ctx else example["response"]
    return example

# labels are "SARCASM" / "NOT_SARCASM"
label2id = {"NOT_SARCASM": 0, "SARCASM": 1}

def map_label(example):
    example["label"] = label2id[example["label"]]
    return example

fig = fig.map(make_text)
fig = fig.map(map_label)

print(fig["train"][0])


Map:   0%|          | 0/9400 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/9400 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

{'label': 1, 'response': "Yeah I mean there's only one gender anyways, women are objects", 'context': ['LPT: If you\'re worried about hurting someone\'s feelings by using the incorrect gender pronoun, play it safe and spare them any unnecessary pain by using the neutral word "it"', 'When gender is unknown he/him is default.'], 'id': None, 'text': 'LPT: If you\'re worried about hurting someone\'s feelings by using the incorrect gender pronoun, play it safe and spare them any unnecessary pain by using the neutral word "it" [CTX] When gender is unknown he/him is default. [RESP] Yeah I mean there\'s only one gender anyways, women are objects'}


In [ ]:
fig_split = fig["train"].train_test_split(test_size=0.1, seed=42)
fig_train = fig_split["train"]
fig_valid = fig_split["test"]


In [ ]:
from datasets import load_dataset, concatenate_datasets, Features, Value

# --- tweet_eval / irony ---
tweet = load_dataset("tweet_eval", "irony")

tweet_train = tweet["train"]
tweet_valid = tweet["validation"]

# Make sure labels are plain int64, not ClassLabel
def to_int_label(example):
    example["label"] = int(example["label"])
    return example

tweet_train = tweet_train.map(to_int_label)
tweet_valid = tweet_valid.map(to_int_label)

# Force features to match: text: string, label: int64
tweet_features = Features({"text": Value("string"), "label": Value("int64")})
tweet_train = tweet_train.cast(tweet_features)
tweet_valid = tweet_valid.cast(tweet_features)

print(tweet_train.features)


Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/955 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2862 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/955 [00:00<?, ? examples/s]

{'text': Value('string'), 'label': Value('int64')}


In [ ]:
train_raw = concatenate_datasets([fig_train, tweet_train])
valid_raw = concatenate_datasets([fig_valid, tweet_valid])

print(train_raw)
print(valid_raw)



Dataset({
    features: ['label', 'response', 'context', 'id', 'text'],
    num_rows: 11322
})
Dataset({
    features: ['label', 'response', 'context', 'id', 'text'],
    num_rows: 1895
})


In [ ]:
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

train_enc = train_raw.map(tokenize_batch, batched=True)
valid_enc = valid_raw.map(tokenize_batch, batched=True)

# rename label -> labels for Trainer
train_enc = train_enc.rename_column("label", "labels")
valid_enc = valid_enc.rename_column("label", "labels")

cols = ["input_ids", "attention_mask", "labels"]
train_enc.set_format(type="torch", columns=cols)
valid_enc.set_format(type="torch", columns=cols)

train_ds = train_enc
eval_ds  = valid_enc


Map:   0%|          | 0/11322 [00:00<?, ? examples/s]

Map:   0%|          | 0/1895 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

labels_np = np.array(train_ds["labels"])
class_counts = np.bincount(labels_np, minlength=2)
total = class_counts.sum()

# Inverse-frequency weights (you can tweak)
weights = total / (2.0 * class_counts.astype(float))
print("Class counts:", class_counts)
print("Class weights:", weights)

import torch
class_weights = torch.tensor(weights, dtype=torch.float)


Class counts: [5638 5684]
Class weights: [1.00407946 0.99595355]


In [ ]:
from transformers import Trainer
import torch.nn as nn

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        if self.class_weights is not None:
            cw = self.class_weights.to(logits.device)
            loss_fct = nn.CrossEntropyLoss(weight=cw)
        else:
            loss_fct = nn.CrossEntropyLoss()

        loss = loss_fct(
            logits.view(-1, model.config.num_labels),
            labels.view(-1)
        )
        if return_outputs:
            return loss, outputs
        return loss


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_DIR, "checkpoints_joint"),
    num_train_epochs=4,               # more epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=1e-5,               # smaller LR
    weight_decay=0.01,
    logging_steps=200,
    save_total_limit=2,
    fp16=True,                        # good for 4070
)


In [ ]:
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    res = {}
    res.update(metric_acc.compute(predictions=preds, references=labels))
    # F1 of sarcasm class (1)
    res.update(metric_f1.compute(predictions=preds, references=labels, average=None, labels=[1]))
    # metric_f1 returns {"f1": [value]} for labels=[1]
    res["f1_sarcasm"] = res.pop("f1")[0]
    return res


In [ ]:
from transformers import Trainer
import torch.nn as nn
import torch

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: int | None = None,
    ):
        # standard pattern: pull out labels
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # choose loss with / without class weights
        if self.class_weights is not None:
            cw = self.class_weights.to(logits.device)
            loss_fct = nn.CrossEntropyLoss(weight=cw)
        else:
            loss_fct = nn.CrossEntropyLoss()

        loss = loss_fct(
            logits.view(-1, model.config.num_labels),
            labels.view(-1),
        )

        if return_outputs:
            return loss, outputs
        return loss


In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
    class_weights=class_weights,
)

trainer.train()

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)


Step,Training Loss
200,0.524300
400,0.528000
600,0.498100
800,0.444600
1000,0.406000
1200,0.420700
1400,0.415900
1600,0.332900
1800,0.336300
2000,0.355700


('D:\\hf_models\\roberta_sarcasm\\tokenizer_config.json',
 'D:\\hf_models\\roberta_sarcasm\\special_tokens_map.json',
 'D:\\hf_models\\roberta_sarcasm\\vocab.json',
 'D:\\hf_models\\roberta_sarcasm\\merges.txt',
 'D:\\hf_models\\roberta_sarcasm\\added_tokens.json',
 'D:\\hf_models\\roberta_sarcasm\\tokenizer.json')